# Database generator

By Kenneth Burchfiel

Released under the MIT License

This script creates tables with fake student data that can be used in visualization and analysis scripts.

In [1]:
import pandas as pd
pd.set_option('display.max_columns', 1000)
from faker import Faker
import numpy as np
import geopandas
import folium
import sqlalchemy

fake = Faker() # https://faker.readthedocs.io/en/master/index.html

In [2]:
rng = np.random.default_rng(seed=1158) 
# https://numpy.org/doc/stable/reference/random/generator.html

In [3]:
# Source of addresses:
# https://nces.ed.gov/programs/edge/geographic/schoollocations

# df_address_list = pd.read_excel('EDGE_GEOCODE_PUBLICSCH_2122.xlsx') # Took
# # around 30 seconds to load
# # Converting to a .csv for faster loading:
# df_address_list.to_csv('school_address_list.csv', index = False)

# df_private_school_address_list = pd.read_excel('EDGE_GEOCODE_PRIVATESCH_1920.xlsx') 
# # Converting to a .csv for faster loading:
# df_private_school_address_list.to_csv('private_school_address_list.csv', index = False)

The NCES has made over 120,000 school addresses and corresponding geographic coordinates publicly available on their website, so I chose to use that data as a source of student addresses.

In [4]:
df_private_school_address_list = pd.read_csv(
    'private_school_address_list.csv', low_memory = False)

df_public_school_address_list = pd.read_csv('school_address_list.csv', 
    low_memory = False)

df_address_list = pd.concat([df_public_school_address_list, 
    df_private_school_address_list])
df_address_list

,NCESSCH,LEAID,NAME,OPSTFIPS,STREET,CITY,STATE,ZIP,STFIP,CNTY,NMCNTY,LOCALE,LAT,LON,CBSA,NMCBSA,CBSATYPE,CSA,NMCSA,NECTA,NMNECTA,CD,SLDL,SLDU,SCHOOLYEAR,PPIN
0,1.000050e+10,100005.0,Albertville Middle School,1.0,600 E Alabama Ave,Albertville,AL,35950,1,1095,Marshall County,32,34.260200,-86.206200,10700,"Albertville, AL",2,N,N,N,N,104,01026,01009,2021-2022,NaN
1,1.000050e+10,100005.0,Albertville High School,1.0,402 E McCord Ave,Albertville,AL,35950,1,1095,Marshall County,32,34.262200,-86.204900,10700,"Albertville, AL",2,N,N,N,N,104,01026,01009,2021-2022,NaN
2,1.000050e+10,100005.0,Albertville Intermediate School,1.0,901 W McKinney Ave,Albertville,AL,35950,1,1095,Marshall County,32,34.273300,-86.220100,10700,"Albertville, AL",2,N,N,N,N,104,01026,01009,2021-2022,NaN
3,1.000050e+10,100005.0,Albertville Elementary School,1.0,145 West End Drive,Albertville,AL,35950,1,1095,Marshall County,32,34.252700,-86.221806,10700,"Albertville, AL",2,N,N,N,N,104,01026,01009,2021-2022,NaN
4,1.000050e+10,100005.0,Albertville Kindergarten and PreK,1.0,257 Country Club Rd,Albertville,AL,35951,1,1095,Marshall County,32,34.289800,-86.193300,10700,"Albertville, AL",2,N,N,N,N,104,01026,01009,2021-2022,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
21567,NaN,NaN,YESHIVA BETH MIKROH,NaN,221 VIOLA RD,MONSEY,NY,10952,36,36087,Rockland County,21,41.135509,-74.078211,35620,"New York-Newark-Jersey City, NY-NJ-PA",1,408,"New York-Newark, NY-NJ-CT-PA",N,N,3617,36098,36038,2019-2020,Y9105109
21568,NaN,NaN,METRO CHRISTIAN ACADEMY,NaN,6363 S TRENTON AVE,TULSA,OK,74136,40,40143,Tulsa County,11,36.069900,-95.965841,46140,"Tulsa, OK",1,538,"Tulsa-Muskogee-Bartlesville, OK",N,N,4001,40071,40035,2019-2020,Y9105326
21569,NaN,NaN,SAYRE SCHOOL,NaN,194 N LIMESTONE,LEXINGTON,KY,40507,21,21067,Fayette County,11,38.047430,-84.493511,30460,"Lexington-Fayette, KY",1,336,"Lexington-Fayette--Richmond--Frankfort, KY",N,N,2106,21077,21013,2019-2020,Z0516931
21570,NaN,NaN,FORT WORTH CHRISTIAN SCHOOL,NaN,6200 HOLIDAY LN,NORTH RICHLAND HILLS,TX,76180,48,48439,Tarrant County,21,32.857358,-97.220487,19100,"Dallas-Fort Worth-Arlington, TX",1,206,"Dallas-Fort Worth, TX-OK",N,N,4812,48091,48009,2019-2020,Z1326859


In [5]:
# Adding leading zeroes back into zip codes:

df_address_list['ZIP'] = df_address_list['ZIP'].astype('str').str.zfill(5)
# Took only 0.4 seconds to load
df_address_list

,NCESSCH,LEAID,NAME,OPSTFIPS,STREET,CITY,STATE,ZIP,STFIP,CNTY,NMCNTY,LOCALE,LAT,LON,CBSA,NMCBSA,CBSATYPE,CSA,NMCSA,NECTA,NMNECTA,CD,SLDL,SLDU,SCHOOLYEAR,PPIN
0,1.000050e+10,100005.0,Albertville Middle School,1.0,600 E Alabama Ave,Albertville,AL,35950,1,1095,Marshall County,32,34.260200,-86.206200,10700,"Albertville, AL",2,N,N,N,N,104,01026,01009,2021-2022,NaN
1,1.000050e+10,100005.0,Albertville High School,1.0,402 E McCord Ave,Albertville,AL,35950,1,1095,Marshall County,32,34.262200,-86.204900,10700,"Albertville, AL",2,N,N,N,N,104,01026,01009,2021-2022,NaN
2,1.000050e+10,100005.0,Albertville Intermediate School,1.0,901 W McKinney Ave,Albertville,AL,35950,1,1095,Marshall County,32,34.273300,-86.220100,10700,"Albertville, AL",2,N,N,N,N,104,01026,01009,2021-2022,NaN
3,1.000050e+10,100005.0,Albertville Elementary School,1.0,145 West End Drive,Albertville,AL,35950,1,1095,Marshall County,32,34.252700,-86.221806,10700,"Albertville, AL",2,N,N,N,N,104,01026,01009,2021-2022,NaN
4,1.000050e+10,100005.0,Albertville Kindergarten and PreK,1.0,257 Country Club Rd,Albertville,AL,35951,1,1095,Marshall County,32,34.289800,-86.193300,10700,"Albertville, AL",2,N,N,N,N,104,01026,01009,2021-2022,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
21567,NaN,NaN,YESHIVA BETH MIKROH,NaN,221 VIOLA RD,MONSEY,NY,10952,36,36087,Rockland County,21,41.135509,-74.078211,35620,"New York-Newark-Jersey City, NY-NJ-PA",1,408,"New York-Newark, NY-NJ-CT-PA",N,N,3617,36098,36038,2019-2020,Y9105109
21568,NaN,NaN,METRO CHRISTIAN ACADEMY,NaN,6363 S TRENTON AVE,TULSA,OK,74136,40,40143,Tulsa County,11,36.069900,-95.965841,46140,"Tulsa, OK",1,538,"Tulsa-Muskogee-Bartlesville, OK",N,N,4001,40071,40035,2019-2020,Y9105326
21569,NaN,NaN,SAYRE SCHOOL,NaN,194 N LIMESTONE,LEXINGTON,KY,40507,21,21067,Fayette County,11,38.047430,-84.493511,30460,"Lexington-Fayette, KY",1,336,"Lexington-Fayette--Richmond--Frankfort, KY",N,N,2106,21077,21013,2019-2020,Z0516931
21570,NaN,NaN,FORT WORTH CHRISTIAN SCHOOL,NaN,6200 HOLIDAY LN,NORTH RICHLAND HILLS,TX,76180,48,48439,Tarrant County,21,32.857358,-97.220487,19100,"Dallas-Fort Worth-Arlington, TX",1,206,"Dallas-Fort Worth, TX-OK",N,N,4812,48091,48009,2019-2020,Z1326859


In [6]:
df_address_list_consolidated = df_address_list[[
    'STREET', 'CITY', 'STATE', 'ZIP', 'LAT', 'LON'
    ]].copy()
df_address_list_consolidated.columns = [
    column.title() for column in df_address_list_consolidated.columns]
df_address_list_consolidated

,Street,City,State,Zip,Lat,Lon
0,600 E Alabama Ave,Albertville,AL,35950,34.260200,-86.206200
1,402 E McCord Ave,Albertville,AL,35950,34.262200,-86.204900
2,901 W McKinney Ave,Albertville,AL,35950,34.273300,-86.220100
3,145 West End Drive,Albertville,AL,35950,34.252700,-86.221806
4,257 Country Club Rd,Albertville,AL,35951,34.289800,-86.193300
...,...,...,...,...,...,...
21567,221 VIOLA RD,MONSEY,NY,10952,41.135509,-74.078211
21568,6363 S TRENTON AVE,TULSA,OK,74136,36.069900,-95.965841
21569,194 N LIMESTONE,LEXINGTON,KY,40507,38.047430,-84.493511
21570,6200 HOLIDAY LN,NORTH RICHLAND HILLS,TX,76180,32.857358,-97.220487


In [7]:
df_address_list_consolidated.dtypes

Street     object
City       object
State      object
Zip        object
Lat       float64
Lon       float64
dtype: object

In [8]:
# Combining each street, city, state, and zip into an 'Address' entry:
df_address_list_consolidated['Address'] = \
(df_address_list_consolidated['Street'] + ', ' + 
df_address_list_consolidated['City'] + ', ' + 
df_address_list_consolidated['State'] + ' ' + 
df_address_list_consolidated['Zip'])
df_address_list_consolidated


,Street,City,State,Zip,Lat,Lon,Address
0,600 E Alabama Ave,Albertville,AL,35950,34.260200,-86.206200,"600 E Alabama Ave, Albertville, AL 35950"
1,402 E McCord Ave,Albertville,AL,35950,34.262200,-86.204900,"402 E McCord Ave, Albertville, AL 35950"
2,901 W McKinney Ave,Albertville,AL,35950,34.273300,-86.220100,"901 W McKinney Ave, Albertville, AL 35950"
3,145 West End Drive,Albertville,AL,35950,34.252700,-86.221806,"145 West End Drive, Albertville, AL 35950"
4,257 Country Club Rd,Albertville,AL,35951,34.289800,-86.193300,"257 Country Club Rd, Albertville, AL 35951"
...,...,...,...,...,...,...,...
21567,221 VIOLA RD,MONSEY,NY,10952,41.135509,-74.078211,"221 VIOLA RD, MONSEY, NY 10952"
21568,6363 S TRENTON AVE,TULSA,OK,74136,36.069900,-95.965841,"6363 S TRENTON AVE, TULSA, OK 74136"
21569,194 N LIMESTONE,LEXINGTON,KY,40507,38.047430,-84.493511,"194 N LIMESTONE, LEXINGTON, KY 40507"
21570,6200 HOLIDAY LN,NORTH RICHLAND HILLS,TX,76180,32.857358,-97.220487,"6200 HOLIDAY LN, NORTH RICHLAND HILLS, TX 76180"


In [9]:
df_va_addresses = df_address_list_consolidated.query("State == 'VA'").copy()
df_va_addresses

,Street,City,State,Zip,Lat,Lon,Address
92946,1000 East Rio Rd,Charlottesville,VA,22901,38.062500,-78.463800,"1000 East Rio Rd, Charlottesville, VA 22901"
92947,105 E Country Club Lane,Covington,VA,24426,37.788133,-79.909846,"105 E Country Club Lane, Covington, VA 24426"
92948,325 Pleasant Valley Rd,Harrisonburg,VA,22801,38.401100,-78.902100,"325 Pleasant Valley Rd, Harrisonburg, VA 22801"
92949,49 Hornet Road,Fishersville,VA,22939,38.116263,-78.989196,"49 Hornet Road, Fishersville, VA 22939"
92950,13400 Woodside Ln,Newport News,VA,23608,37.171400,-76.527000,"13400 Woodside Ln, Newport News, VA 23608"
...,...,...,...,...,...,...,...
21401,8918 TIDEWATER DR,NORFOLK,VA,23503,36.939590,-76.254131,"8918 TIDEWATER DR, NORFOLK, VA 23503"
21481,1071 WOODBERRY SQ PL,LYNCHBURG,VA,24502,37.388480,-79.242011,"1071 WOODBERRY SQ PL, LYNCHBURG, VA 24502"
21482,11579 CEDAR CHASE RD,HERNDON,VA,20170,38.997910,-77.344201,"11579 CEDAR CHASE RD, HERNDON, VA 20170"
21483,1030 BIBLE LN,VINTON,VA,24179,37.286660,-79.834681,"1030 BIBLE LN, VINTON, VA 24179"


In [10]:
# Creating a GeoDataFrame so that we can visualize each address on a map:
gdf_va_addresses = geopandas.GeoDataFrame(df_va_addresses, 
geometry = geopandas.points_from_xy(df_va_addresses['Lon'], 
df_va_addresses['Lat']))
# See https://geopandas.org/en/stable/gallery/create_geopandas_from_pandas.html
gdf_va_addresses

,Street,City,State,Zip,Lat,Lon,Address,geometry
92946,1000 East Rio Rd,Charlottesville,VA,22901,38.062500,-78.463800,"1000 East Rio Rd, Charlottesville, VA 22901",POINT (-78.46380 38.06250)
92947,105 E Country Club Lane,Covington,VA,24426,37.788133,-79.909846,"105 E Country Club Lane, Covington, VA 24426",POINT (-79.90985 37.78813)
92948,325 Pleasant Valley Rd,Harrisonburg,VA,22801,38.401100,-78.902100,"325 Pleasant Valley Rd, Harrisonburg, VA 22801",POINT (-78.90210 38.40110)
92949,49 Hornet Road,Fishersville,VA,22939,38.116263,-78.989196,"49 Hornet Road, Fishersville, VA 22939",POINT (-78.98920 38.11626)
92950,13400 Woodside Ln,Newport News,VA,23608,37.171400,-76.527000,"13400 Woodside Ln, Newport News, VA 23608",POINT (-76.52700 37.17140)
...,...,...,...,...,...,...,...,...
21401,8918 TIDEWATER DR,NORFOLK,VA,23503,36.939590,-76.254131,"8918 TIDEWATER DR, NORFOLK, VA 23503",POINT (-76.25413 36.93959)
21481,1071 WOODBERRY SQ PL,LYNCHBURG,VA,24502,37.388480,-79.242011,"1071 WOODBERRY SQ PL, LYNCHBURG, VA 24502",POINT (-79.24201 37.38848)
21482,11579 CEDAR CHASE RD,HERNDON,VA,20170,38.997910,-77.344201,"11579 CEDAR CHASE RD, HERNDON, VA 20170",POINT (-77.34420 38.99791)
21483,1030 BIBLE LN,VINTON,VA,24179,37.286660,-79.834681,"1030 BIBLE LN, VINTON, VA 24179",POINT (-79.83468 37.28666)


## Visualizing the addresses:

In [11]:
m = folium.Map(location=[38, -79], zoom_start = 7, tiles='Stamen Terrain') 
# https://github.com/python-visualization/folium/blob/main/examples/Quickstart.ipynb

for i in range(len(gdf_va_addresses)):
    coords = gdf_va_addresses.iloc[i]['geometry']
    address = gdf_va_addresses.iloc[i]['Address']
    # print(coords)
    folium.CircleMarker(
        location = [coords.y, coords.x],
        radius = 5,
        tooltip= address,
        fill = True).add_to(m)
    # For the use of coords.y and coords.x, see:
    # https://shapely.readthedocs.io/en/stable/reference/shapely.Point.html

m

## Filing in student information with Faker and np.random:

In [12]:
student_id_column = [i for i in range(1, 4001)]
male_first_name_list = [fake.first_name_male() for i in range(2000)]
# See https://faker.readthedocs.io/en/master/providers/faker.providers.person.html
female_first_name_list = [fake.first_name_female() for i in range(2000)]
first_name_column = male_first_name_list + female_first_name_list
gender_column = ['Male' for i in range(2000)] + ['Female' for i in range(2000)]
school_column = [rng.choice([
'Sycamore Academy', 'Dogwood Academy', 'Chestnut Academy', 'Hickory Academy'
]) for i in range(4000)]
# Creating a school acronym column by splitting each school name, 
# then combining the first letter of each word together:
acronym_column = [school.split(' ')[0][0] + school.split(' ')[1][0]
for school in school_column]
last_name_column = [fake.last_name() for i in range(4000)]
grade_column = [rng.choice([
'K', '1', '2', '3', '4', '5', '6', '7', '8', '9', '10', '11', '12'
]) for i in range(4000)]
race_column = [rng.choice(['African American', 'American Indian', 
'Asian', 'White'], p = [0.3, 0.05, 0.2, 0.45]) for i in range(4000)]
ethnicity_column = [rng.choice(['Hispanic', 'Non-Hispanic'], 
p = [0.3, 0.7]) for i in range(4000)]

## Converting these columns into a DataFrame:

In [13]:
df_curr_enrollment = pd.DataFrame({'Student_ID':student_id_column,
'First_Name':first_name_column, 'Last_Name':last_name_column, 
'Full_School_Name':school_column, 'School':acronym_column, 'Grade':grade_column, 
'Gender':gender_column, 'Race':race_column, 'Ethnicity':ethnicity_column})
df_curr_enrollment

,Student_ID,First_Name,Last_Name,Full_School_Name,School,Grade,Gender,Race,Ethnicity
0,1,Daniel,Zimmerman,Chestnut Academy,CA,9,Male,White,Non-Hispanic
1,2,James,House,Sycamore Academy,SA,10,Male,African American,Hispanic
2,3,Anthony,Brown,Hickory Academy,HA,6,Male,African American,Non-Hispanic
3,4,Michael,Lee,Chestnut Academy,CA,6,Male,White,Hispanic
4,5,Brandon,Riddle,Chestnut Academy,CA,8,Male,White,Hispanic
...,...,...,...,...,...,...,...,...,...
3995,3996,Kelsey,Ball,Sycamore Academy,SA,9,Female,African American,Non-Hispanic
3996,3997,Leslie,Maxwell,Sycamore Academy,SA,4,Female,African American,Non-Hispanic
3997,3998,Heather,Myers,Hickory Academy,HA,2,Female,Asian,Non-Hispanic
3998,3999,Ebony,Cox,Sycamore Academy,SA,7,Female,Asian,Non-Hispanic


I'll now merge in address info by:
1. Duplicating the list so that there are enough addresses for each student
1. Assigning a student_id value to each row of df_va_addresses
1. Merging the duplicated address list into the student list

In [14]:
df_va_addresses_for_merge = pd.concat([df_va_addresses, df_va_addresses])
df_va_addresses_for_merge['Student_ID'] = [i for i in range(len(df_va_addresses_for_merge))]
df_va_addresses_for_merge

,Street,City,State,Zip,Lat,Lon,Address,geometry,Student_ID
92946,1000 East Rio Rd,Charlottesville,VA,22901,38.062500,-78.463800,"1000 East Rio Rd, Charlottesville, VA 22901",POINT (-78.46380 38.06250),0
92947,105 E Country Club Lane,Covington,VA,24426,37.788133,-79.909846,"105 E Country Club Lane, Covington, VA 24426",POINT (-79.90985 37.78813),1
92948,325 Pleasant Valley Rd,Harrisonburg,VA,22801,38.401100,-78.902100,"325 Pleasant Valley Rd, Harrisonburg, VA 22801",POINT (-78.90210 38.40110),2
92949,49 Hornet Road,Fishersville,VA,22939,38.116263,-78.989196,"49 Hornet Road, Fishersville, VA 22939",POINT (-78.98920 38.11626),3
92950,13400 Woodside Ln,Newport News,VA,23608,37.171400,-76.527000,"13400 Woodside Ln, Newport News, VA 23608",POINT (-76.52700 37.17140),4
...,...,...,...,...,...,...,...,...,...
21401,8918 TIDEWATER DR,NORFOLK,VA,23503,36.939590,-76.254131,"8918 TIDEWATER DR, NORFOLK, VA 23503",POINT (-76.25413 36.93959),5511
21481,1071 WOODBERRY SQ PL,LYNCHBURG,VA,24502,37.388480,-79.242011,"1071 WOODBERRY SQ PL, LYNCHBURG, VA 24502",POINT (-79.24201 37.38848),5512
21482,11579 CEDAR CHASE RD,HERNDON,VA,20170,38.997910,-77.344201,"11579 CEDAR CHASE RD, HERNDON, VA 20170",POINT (-77.34420 38.99791),5513
21483,1030 BIBLE LN,VINTON,VA,24179,37.286660,-79.834681,"1030 BIBLE LN, VINTON, VA 24179",POINT (-79.83468 37.28666),5514


In [15]:
df_curr_enrollment = df_curr_enrollment.merge(df_va_addresses_for_merge, on = 'Student_ID', how = 'left')
df_curr_enrollment.sort_values(['School', 'Grade', 'Last_Name', 'First_Name'], inplace = True)
df_curr_enrollment['Students'] = 1 # Useful for pivot tables
# I encountered difficulties trying to upload the 'geometry' column to my
# SQL database, so I'll remove it here.
df_curr_enrollment.drop('geometry', axis = 1, inplace = True)
df_curr_enrollment['Grade_for_Sorting'] = df_curr_enrollment['Grade'].copy()
df_curr_enrollment['Grade_for_Sorting'].replace({'K':'0'}, inplace = True)
df_curr_enrollment['Grade_for_Sorting'] = df_curr_enrollment['Grade_for_Sorting'].astype('int')

df_curr_enrollment

,Student_ID,First_Name,Last_Name,Full_School_Name,School,Grade,Gender,Race,Ethnicity,Street,City,State,Zip,Lat,Lon,Address,Students,Grade_for_Sorting
3872,3873,Jessica,Acosta,Chestnut Academy,CA,1,Female,White,Hispanic,340 N Maple Ave,Purcellville,VA,20132,39.140600,-77.701700,"340 N Maple Ave, Purcellville, VA 20132",1,1
1694,1695,Gary,Alexander,Chestnut Academy,CA,1,Male,White,Hispanic,1616 Nineteenth Street NW,Roanoke,VA,24017,37.293200,-79.966400,"1616 Nineteenth Street NW, Roanoke, VA 24017",1,1
1880,1881,John,Alexander,Chestnut Academy,CA,1,Male,African American,Hispanic,2000 Parkway Blvd,Stafford,VA,22554,38.464900,-77.448900,"2000 Parkway Blvd, Stafford, VA 22554",1,1
3939,3940,Melissa,Alexander,Chestnut Academy,CA,1,Female,African American,Non-Hispanic,22325 Mooreview Parkway,Ashburn,VA,20148,38.999300,-77.505500,"22325 Mooreview Parkway, Ashburn, VA 20148",1,1
2672,2673,Jennifer,Barnes,Chestnut Academy,CA,1,Female,White,Hispanic,4800 MATTIE MOORE CT,FAIRFAX,VA,22030,38.839150,-77.347901,"4800 MATTIE MOORE CT, FAIRFAX, VA 22030",1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3586,3587,Lisa,Watkins,Sycamore Academy,SA,K,Female,American Indian,Non-Hispanic,220 Rosa Lane,Winchester,VA,22602,39.126400,-78.146300,"220 Rosa Lane, Winchester, VA 22602",1,0
3963,3964,Sarah,Welch,Sycamore Academy,SA,K,Female,Asian,Non-Hispanic,2035 Spotswood Trail,Louisa,VA,23093,38.075400,-78.100100,"2035 Spotswood Trail, Louisa, VA 23093",1,0
459,460,Charles,Williams,Sycamore Academy,SA,K,Male,White,Non-Hispanic,1701 Evergreen East Pkwy.,Midlothian,VA,23114,37.463200,-77.631600,"1701 Evergreen East Pkwy., Midlothian, VA 23114",1,0
2815,2816,Rhonda,Williams,Sycamore Academy,SA,K,Female,Asian,Hispanic,515 N 10th Street,Richmond,VA,23219,37.542500,-77.430200,"515 N 10th Street, Richmond, VA 23219",1,0


In [16]:
df_curr_enrollment.to_csv('curr_enrollment.csv', index = False)

# Exporting this table to an ElephantSQL database instance:

## Connecting to the database:

In [17]:
with open ("../key_paths/path_to_keys_folder.txt") as file:
    key_path = file.read()

with open(key_path+"/elephantsql_dashschooldemodb_url.txt") as file:
    db_url = file.read()
    # This code reads in my database's URL, which is listed on the home page for my database within elephantsql.com. As shown below, SQLAlchemy can use this URL to connect to the database. 

elephantsql_db_url_for_sqlalchemy = db_url.replace(
    'postgres://', 'postgresql://')
# This change, which is required for SQLAlchemy to work correctly, is based on the code suggested at:
# # https://help.heroku.com/ZKNTJQSK/why-is-sqlalchemy-1-4-x-not-connecting-to-heroku-postgres

elephantsql_engine = sqlalchemy.create_engine(
    elephantsql_db_url_for_sqlalchemy)

Uploading df_curr_enrollment_to_SQL:

In [18]:
df_curr_enrollment.to_sql("curr_enrollment",if_exists = 'replace', 
con = elephantsql_engine, index = False)

1000

Ensuring that the upload worked as expected:

In [19]:
pd.read_sql("curr_enrollment", con = elephantsql_engine)

,Student_ID,First_Name,Last_Name,Full_School_Name,School,Grade,Gender,Race,Ethnicity,Street,City,State,Zip,Lat,Lon,Address,Students,Grade_for_Sorting
0,3873,Jessica,Acosta,Chestnut Academy,CA,1,Female,White,Hispanic,340 N Maple Ave,Purcellville,VA,20132,39.140600,-77.701700,"340 N Maple Ave, Purcellville, VA 20132",1,1
1,1695,Gary,Alexander,Chestnut Academy,CA,1,Male,White,Hispanic,1616 Nineteenth Street NW,Roanoke,VA,24017,37.293200,-79.966400,"1616 Nineteenth Street NW, Roanoke, VA 24017",1,1
2,1881,John,Alexander,Chestnut Academy,CA,1,Male,African American,Hispanic,2000 Parkway Blvd,Stafford,VA,22554,38.464900,-77.448900,"2000 Parkway Blvd, Stafford, VA 22554",1,1
3,3940,Melissa,Alexander,Chestnut Academy,CA,1,Female,African American,Non-Hispanic,22325 Mooreview Parkway,Ashburn,VA,20148,38.999300,-77.505500,"22325 Mooreview Parkway, Ashburn, VA 20148",1,1
4,2673,Jennifer,Barnes,Chestnut Academy,CA,1,Female,White,Hispanic,4800 MATTIE MOORE CT,FAIRFAX,VA,22030,38.839150,-77.347901,"4800 MATTIE MOORE CT, FAIRFAX, VA 22030",1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3995,3587,Lisa,Watkins,Sycamore Academy,SA,K,Female,American Indian,Non-Hispanic,220 Rosa Lane,Winchester,VA,22602,39.126400,-78.146300,"220 Rosa Lane, Winchester, VA 22602",1,0
3996,3964,Sarah,Welch,Sycamore Academy,SA,K,Female,Asian,Non-Hispanic,2035 Spotswood Trail,Louisa,VA,23093,38.075400,-78.100100,"2035 Spotswood Trail, Louisa, VA 23093",1,0
3997,460,Charles,Williams,Sycamore Academy,SA,K,Male,White,Non-Hispanic,1701 Evergreen East Pkwy.,Midlothian,VA,23114,37.463200,-77.631600,"1701 Evergreen East Pkwy., Midlothian, VA 23114",1,0
3998,2816,Rhonda,Williams,Sycamore Academy,SA,K,Female,Asian,Hispanic,515 N 10th Street,Richmond,VA,23219,37.542500,-77.430200,"515 N 10th Street, Richmond, VA 23219",1,0


# Adding a test results table:

We'll next create a table with made-up test results for the fall and spring semesters. The fall results will be random, but the spring results will show more improvement for some schools and grades than others.

In [20]:
df_fall_test_results = df_curr_enrollment.copy()[
    ['Student_ID', 'School', 'Grade']]
df_fall_test_results['Starting_Year'] = 2023 # Starting school year 
df_fall_test_results['Period'] = 'Fall'
df_fall_test_results['Score'] = [round(
    rng.normal(50, 10)) for i in range(len(df_fall_test_results))]
# rng.normal() is used so that the scores can fall (no pun intended!) along 
# a normal distribution. See 
# https://numpy.org/doc/stable/reference/random/generated/numpy.random.Generator.normal.html
df_fall_test_results

,Student_ID,School,Grade,Starting_Year,Period,Score
3872,3873,CA,1,2023,Fall,47
1694,1695,CA,1,2023,Fall,49
1880,1881,CA,1,2023,Fall,57
3939,3940,CA,1,2023,Fall,63
2672,2673,CA,1,2023,Fall,51
...,...,...,...,...,...,...
3586,3587,SA,K,2023,Fall,51
3963,3964,SA,K,2023,Fall,57
459,460,SA,K,2023,Fall,57
2815,2816,SA,K,2023,Fall,57


In [21]:
df_fall_test_results['School'].value_counts()

HA    1038
SA    1021
DA     977
CA     964
Name: School, dtype: int64

In [22]:
rng.random()*10

5.051372429565646

We'll begin initializing the spring results in essentially the same way we did the fall results, but to make the resulting graphs more interesting, we'll have certain grades and schools show notable score changes.

In [23]:
df_spring_test_results = df_curr_enrollment.copy()[
    ['Student_ID', 'School', 'Grade']]
df_spring_test_results['Starting_Year'] = 2023 # Starting school year 
df_spring_test_results['Period'] = 'Spring'

df_spring_test_results['Score'] = [
    (rng.normal(50, 10)) for i in range(len(df_spring_test_results))]

# The following line gives Chestnut and Sycamore Academy students a boost of
# 0-10 points:
df_spring_test_results['Score'] = np.where(
    df_spring_test_results['School'].str.contains('CA|SA', regex = True), 
    df_spring_test_results['Score'] + rng.random()*10, 
    df_spring_test_results['Score'])

# Meanwhile, the following line decreases the scores of 2nd, 4th, and 11th 
# graders anywhere between 0 and 5 points.
df_spring_test_results['Score'] = np.where(
    df_spring_test_results['Grade'].str.contains('2|4|11', regex = True), 
    df_spring_test_results['Score'] - rng.random()*5, 
    df_spring_test_results['Score'])

df_spring_test_results['Score'] = round(
    df_spring_test_results['Score'], 0).astype('int')


# For str.contains(), see: https://pandas.pydata.org/docs/reference/api/pandas.Series.str.contains.html
# rng.normal() is used so that the scores can fall along a normal distribution. 
# See https://numpy.org/doc/stable/reference/random/generated/numpy.random.Generator.normal.html#numpy.random.Generator.normal 
df_spring_test_results

,Student_ID,School,Grade,Starting_Year,Period,Score
3872,3873,CA,1,2023,Spring,62
1694,1695,CA,1,2023,Spring,47
1880,1881,CA,1,2023,Spring,50
3939,3940,CA,1,2023,Spring,52
2672,2673,CA,1,2023,Spring,58
...,...,...,...,...,...,...
3586,3587,SA,K,2023,Spring,58
3963,3964,SA,K,2023,Spring,50
459,460,SA,K,2023,Spring,60
2815,2816,SA,K,2023,Spring,52


In [24]:
df_test_results = pd.concat([df_fall_test_results, df_spring_test_results])
df_test_results

,Student_ID,School,Grade,Starting_Year,Period,Score
3872,3873,CA,1,2023,Fall,47
1694,1695,CA,1,2023,Fall,49
1880,1881,CA,1,2023,Fall,57
3939,3940,CA,1,2023,Fall,63
2672,2673,CA,1,2023,Fall,51
...,...,...,...,...,...,...
3586,3587,SA,K,2023,Spring,58
3963,3964,SA,K,2023,Spring,50
459,460,SA,K,2023,Spring,60
2815,2816,SA,K,2023,Spring,52


Here's a comparison of fall and spring scores for each school-grade pair. Note that Chestnut Academy and Sycamore Academy scores have increased significantly, and that scores for 2nd, 4th, and 11th-grade students often showed declines in their achievement.

In [25]:
df_test_results.pivot_table(index = ['School', 'Grade'], 
columns = 'Period', values = 'Score', aggfunc = 'mean')

Period             Fall     Spring
School Grade                      
CA     1      49.450704  59.084507
       10     50.265823  61.227848
       11     48.986667  55.986667
       12     51.642857  58.757143
       2      51.197368  56.368421
       3      47.819672  59.475410
       4      49.741176  55.988235
       5      48.909091  57.924242
       6      50.094595  60.040541
       7      49.292308  57.907692
       8      49.066667  60.066667
       9      48.961039  59.285714
       K      49.244444  58.033333
DA     1      50.642857  47.803571
       10     48.733333  49.122222
       11     49.757143  48.171429
       12     50.360000  48.173333
       2      51.026667  47.466667
       3      49.354430  52.126582
       4      50.513514  47.486486
       5      49.640449  48.213483
       6      48.114286  52.171429
       7      51.294118  50.661765
       8      50.426667  49.640000
       9      48.015873  50.269841
       K      48.817204  49.236559
HA     1      50.619048  50.488095
       10     52.341463  50.304878
       11     48.415385  44.446154
       12     50.828571  46.085714
       2      46.537634  48.774194
       3      50.366197  50.070423
       4      48.141026  48.192308
       5      48.974359  48.858974
       6      51.468354  49.544304
       7      49.881720  50.322581
       8      50.034483  49.517241
       9      52.452381  51.666667
       K      50.297297  49.648649
SA     1      51.341176  60.529412
       10     48.290698  58.569767
       11     51.120000  57.466667
       12     52.108434  57.927711
       2      50.687500  55.712500
       3      50.371134  59.391753
       4      51.415730  55.191011
       5      49.602740  57.602740
       6      50.671429  59.128571
       7      47.472222  58.069444
       8      49.147059  59.367647
       9      50.042254  58.704225
       K      48.861111  57.597222

We'll now export df_test_results to both a local .csv file and to our SQL database:

In [26]:
df_test_results.to_csv('test_results.csv', index = False)
df_test_results.to_sql("test_results",if_exists = 'replace', 
con = elephantsql_engine, index = False)

1000